# Generate configuration files for each chip

The experiment contains the following chips:

* Chip1 (k1)
* Chip2 (k1)

From the template configuration file, we need to change the following for each chip: 
- image[base_path] -> append Chip# subdirectory
- image[names] -> Chip#

And add in the following from generator
- image[well_start_image]
- image[well_start_xy]
- well_mask

## Generate configuration files

In [1]:
import yaml 
import copy
from os import path, mkdir

In [2]:
with open('../config/config24.yml', 'r') as ymlfile:
    template = yaml.load(ymlfile)

with open('../config/generator.yml', 'r') as ymlfile:
    generator = yaml.load(ymlfile)
    
print yaml.dump(template)
print yaml.dump(generator)

barcodes:
  cluster:
    eps: 0.01
    min_samples: 10
    offset:
    - 600
    - 600
    - 600
    points_to_cluster: 2000
  path: ../Chip24_data/Notes.xlsx
image:
  base_path: ../Chip24_data/
  bugs: 0
  dyes:
  - 1
  - 3
  - 2
  names:
    premerge: Chip24_premerge
    t0: Chip24_t0
    t1: Chip24_t1
    t2: Chip24_t2
  overlap: 0.05
  pixel_size: 6.5
  size: 1024
  well_start_image:
  - 1
  - 1
  well_start_xy:
  - 518
  - 434
well_mask:
  filename: ../Well Masks/k2_mask.tif
  pixel_size: 8.33625
  well_start_xy:
  - 242
  - 378

Chip1:
  image:
    well_start_image:
    - 1
    - 1
    well_start_xy:
    - 350
    - 350
  well_mask:
    filename: ../Well Masks/k1_mask.tif
    pixel_size: 8.33625
    well_start_xy:
    - 182
    - 202
Chip2:
  image:
    well_start_image:
    - 1
    - 1
    well_start_xy:
    - 525
    - 320
  well_mask:
    filename: ../Well Masks/k1_mask.tif
    pixel_size: 8.33625
    well_start_xy:
    - 182
    - 202



/Users/mandy/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  
/Users/mandy/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [3]:
# Create base analysis directory
base = '/Users/mandy/Develop/PyCharm/BPCP/kchip/Accessory/Chip24_data/Image Data'

if not path.exists(base):
    mkdir(base)
    
# Create directories
for chip in generator.keys():

    if not path.exists(path.join(base,chip)):
        mkdir(path.join(base,chip))

    if not path.exists(path.join(base,chip)):
        mkdir(path.join(base,chip))

In [4]:
for chip in generator.keys():

    config = copy.deepcopy(template)

    try: 
        config['image']['base_path'] = path.join(config['image']['base_path'],chip)

        for name in config['image']['names'].keys():
            config['image']['names'][name] = config['image']['names'][name].replace('Chip1',chip)

        config['image']['well_start_image'] = generator[chip]['image']['well_start_image']
        config['image']['well_start_xy'] = generator[chip]['image']['well_start_xy']
        config['well_mask'] = generator[chip]['well_mask']

        with open(path.join(base,chip,'config.yml'), 'w') as outfile:
            yaml.dump(config, outfile,default_flow_style=False)
    except:
        print 'Error on:', chip

## Setup jupyter notebooks for each chip

Use Analysis_template.ipynb as template

In [5]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

def create_notebook(template):
    ''' Create output notebook from template '''

    with open(template) as f:
        nb = nbformat.read(f, as_version=4)

    return nb

def execute_notebook(nb):
    ''' Execute notebook (returns in place) kernel=python2'''
    ep = ExecutePreprocessor(timeout=600, kernel_name='python2')
    ep.preprocess(nb,{'metadata': {'path': './'}})

def write_notebook(nb,output_path):
    ''' Write notebok file '''
    with open(output_path, 'wt') as f:
        nbformat.write(nb, f)


In [6]:
template = '/Users/mandy/Develop/PyCharm/BPCP/kchip/Accessory/Notebooks/Analysis_Template.ipynb'
from os import chdir

for chip in generator.keys():
    print chip

    print 'Creating...'
    # Create from template
    nb = create_notebook(template)

    print 'Moving to... '
    chdir(path.join(base,chip))

    print 'Executing...'

    try: 
        # Execute
        execute_notebook(nb)

    except Exception as e:
        print 'Error executing.'
        print e

    print 'Writing...'
    # Write
    outfile = 'Analysis.ipynb'
    write_notebook(nb,outfile)

Chip2
Creating...
Moving to... 
Executing...
Error executing.
An error occurred while executing the following cell:
------------------
# basic imports 
import yaml
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Append top level directory with kchip package
import sys
sys.path.append('../../../')

# kchip imports
import kchip.io as kchip_io
import kchip.analyze as kchip_analyze

# Other
%matplotlib inline

# Read in config file
with open('../config/config24.yml', 'r') as ymlfile:
    config = yaml.load(ymlfile)
    
print yaml.dump(config)
------------------

---------------------------------------------------------------------------
ImportError                               Traceback (most recent call last)
<ipython-input-1-ecc8e55342ed> in <module>()
     10 
     11 # kchip imports
---> 12 import kchip.io as kchip_io
     13 import kchip.analyze as kchip_analyze
     14 

ImportError: No module named kchip.io
ImportError: No module named kchip.io

Writing..